In [1]:
import numpy as np
import pandas as pd
import time
from ema_workbench.analysis import clusterer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from PyDICE_t2xco2 import PyDICE
import os
os.chdir(os.getcwd())

from specify import specify_levers
from ema_workbench import (save_results)
from ema_workbench import (perform_experiments, Model, Policy, RealParameter, IntegerParameter, ScalarOutcome,
                           ema_logging, MultiprocessingEvaluator)

ema_logging.log_to_stderr(ema_logging.INFO)

model = PyDICE()
dice_sm = Model('dicesmEMA', function = model)
dice_opt = pd.read_excel("DICE2013R.xlsm" ,sheet_name = "Opttax", index_col = 0)

dice_sm.uncertainties = [IntegerParameter('t2xco2_index', 0, 999),
                         IntegerParameter('t2xco2_dist',0 , 2),
                         IntegerParameter('fdamage', 0, 2),
                         RealParameter('tfp_gr', 0.07, 0.09),
                         RealParameter('sigma_gr', -0.012, -0.008),
                         RealParameter('pop_gr', 0.1, 0.15),
                         RealParameter('fosslim',  4000.0, 13649),
                         IntegerParameter('cback', 100, 600)]

dice_sm.levers = [RealParameter('sr', 0.1, 0.5),
                  RealParameter('irstp',  0.001, 0.015),
                  IntegerParameter('periodfullpart', 10, 58),
                  IntegerParameter('miu_period', 10, 58)]


dice_sm.outcomes = [ScalarOutcome('ECS', ScalarOutcome.MAXIMIZE),                    
                    ScalarOutcome('Utility 2300', ScalarOutcome.MAXIMIZE)]
n_scenarios=180000
nord_optimal_policy = Policy('nord_optimal_policy', **specify_levers(np.mean(dice_opt.iloc[129]),0.015,0,29))


start = time.time()
with MultiprocessingEvaluator(dice_sm) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=nord_optimal_policy)
end = time.time()


C:\Users\Shajee\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Shajee\Anaconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)
C:\Users\Shajee\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Shajee\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Shajee\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from nump

In [2]:
experiments, outcomes = results

In [3]:
outcomesDF = pd.DataFrame(outcomes)

In [4]:
resDF = (experiments.T.append(outcomesDF.T)).T

In [5]:
len(np.unique(resDF[(resDF.t2xco2_dist == 0)]["t2xco2_index"] == 1000))

1

In [6]:
list_exp = [[],[],[]]
for i in range(3):
    list_exp[i] = np.array(resDF[(resDF.t2xco2_dist == i)]["ECS"]).astype(float)

In [25]:
import numpy as np
from scipy.stats import norm, lognorm, cauchy, wilcoxon
import json


with open('ecs_dist_v5.json') as f:
    d=json.load(f)

np.random.seed(10)

minb = 0
maxb = 20
nsamples = 60000

samples_norm = np.zeros((0,))
while samples_norm.shape[0] < nsamples:
    samples = (norm.rvs(d['norm'][0],d['norm'][1],nsamples))
    accepted = samples[(samples >= minb) & (samples <= maxb)]
    samples_norm = np.concatenate((samples_norm, accepted), axis=0)
samples_norm = samples_norm[:nsamples]

samples_lognorm = np.zeros((0,))
while samples_lognorm.shape[0] < nsamples:
    samples = (lognorm.rvs(d['lognorm'][0],d['lognorm'][1],d['lognorm'][2],nsamples))
    accepted = samples[(samples >= minb) & (samples <= maxb)]
    samples_lognorm = np.concatenate((samples_lognorm, accepted), axis=0)
samples_lognorm = samples_lognorm[:nsamples]

samples_cauchy = np.zeros((0,))
while samples_cauchy.shape[0] < nsamples:
    samples = (cauchy.rvs(d['cauchy'][0],d['cauchy'][1],nsamples))
    accepted = samples[(samples >= minb) & (samples <= maxb)]
    samples_cauchy = np.concatenate((samples_cauchy, accepted), axis=0)
samples_cauchy = samples_cauchy[:nsamples]

In [26]:
list_samples = [samples_norm, samples_lognorm, samples_cauchy]

### Entropy Comparison of Sampling Techniques

In [34]:
from scipy import stats

In [35]:
for i in range(3):
    x = ["Normal","Log-Normal","Cauchy"]
    list_samples[i].sort()
    list_exp[i].sort()
    y = stats.entropy(list_samples[i], list_exp[i])
    print(str(x[i])+": "+ str(y))

Normal: 0.00032349888222737945
Log-Normal: 0.0002984975618048922
Cauchy: 0.0008619169349033456


In [36]:
descDF = pd.DataFrame()
for i in range(3):
    x = ["Normal","Log-Normal","Cauchy"]
    sampleDF = pd.DataFrame(list_samples[i], columns=[str(x[i])+" Sampled Set"])
    descDF = pd.concat([descDF,sampleDF], axis=1)
    expDF = pd.DataFrame(list_exp[i], columns=[str(x[i])+" Experiment Set"])
    descDF = pd.concat([descDF,expDF], axis=1)

In [37]:
descDF.describe(percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,Normal Sampled Set,Normal Experiment Set,Log-Normal Sampled Set,Log-Normal Experiment Set,Cauchy Sampled Set,Cauchy Experiment Set
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000
mean,3.239672,3.201974,3.296763,3.257213,3.535251,3.406725
std,1.409426,1.339230,1.524329,1.516858,2.136046,1.945970
min,0.000676,0.033768,0.000436,0.095995,0.000059,0.024124
1%,0.299472,0.295406,0.567992,0.461744,0.335904,0.315314
5%,0.946305,1.027264,1.151042,1.061560,1.218684,1.237627
10%,1.400381,1.474526,1.512491,1.450988,1.794850,1.859637
25%,2.237658,2.287821,2.200395,2.173841,2.556688,2.540184
50%,3.211309,3.187206,3.115868,3.101255,3.164942,3.121279
75%,4.192123,4.080875,4.181740,4.158097,3.868967,3.723525
